# Modeule 1, Example 2: Power Market Three-Stage Recourse Problem

**Petrobras Course on Stochastic Optimization**\
Instructors: Steve Gabriel \& Dominic Flocco\
Date: October 2024
## Step 0: Install Code Dependencies

Before we get started, run the following cells to load the necessary Python packages for the notebook and some helper functions we'll use to display model solutions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.mpec import *
pd.set_option('display.float_format', '{:.2f}'.format)

# set solver
opt = SolverFactory('glpk')

In [ ]:
# helper function to pprint solution after stoc_model solve
def print_stoc_solution(stoc_model):
    print(f"============== {stoc_model.name} Solution Report ===============")
    print(f"\n***** REVENUE = ${stoc_model.objective():.2f} ")

    # store model results in dictionary
    result_dict = dict()
    for c in stoc_model.contracts:
        result_dict[f"Contract {c}"] = [stoc_model.contract_sale[c,s].value for s in stoc_model.scenarios]
    for p in stoc_model.periods:
        result_dict[f"Pool {p}"] = [stoc_model.pool_sale[p,s].value for s in stoc_model.scenarios]

    # convert dictionary to Padnas dataframe
    result_df = pd.DataFrame().from_dict(result_dict,
                                         orient='index',
                                         columns=[f"Scenario {s}" for s in stoc_model.scenarios])

    return result_df

## Problem Description
Adapted from: Conejo, A. J., Carrión, M., & Morales, J. M. (2010). *Decision making under uncertainty in electricity markets* (Vol. 1, pp. 376-384). New York: Springer. Chapter 2.

### Problem Statement
Consider an electricity producer with a production capacity of 120 MW who is facing both uncertain electricty demand and price for the next week. This producer needs to determin its selling strategy during a future market horizon of 6 days, divideded into two 3-day periods. The producer has the folloing bilateral contracts available:
1) A selling contract of up to 50 MW at \\$25/MHw at the beginning of the 6-day period and spanning the entire 6-day period.
2) A selling contract of up to 40 MW at \\$26/HWh just after the first 3-day period and covering the last 3-day period.
More concretely, the contract data for the producer is given in the table below.

| Contract | Duration | Hours | Price (\$/MWh) | Power Cap. (MW)|
| -------- | -------- | ----- | ------------- | ---------------|
| A | Entire 6-day period | 144 | 25 | 50 |
| B | 2nd 3-day period | 72| 26| 40|

Complementarily to bilateral contracting, the producer can sell energy in the poolat a constant power during the first and second 3-day periods. The pool prices scenarios are:

1. Pool prices for the first 3-day period are \\$27/MHw with probability 0.4 and \\$23/MWh with probability 0.6
2. If the pice during the first 3-day period is \\$27/MWh, prices during the second 3-day period are \\$28/MWh with probability 0.4 and \\$26/MWh with probability 0.6.
3. Alternatively, if the price during the first 3-day period is \\$23/MWh, prices during the second 2-day period are \\$24/MWh with probability 0.4 and \\$22/MWh with probability 0.6

The scenaro data for the producer is given in the following table:

| Scenario | 1st 2-day price (\\$/MWh) | 2nd 3-day price (\\$/MWh) | Probability (per unit)|
| - | - | - | - |
| 1 | 27 | 28 | $0.4\times 0.4= 0.16$|
| 2 | 27 | 26 | $0.4\times 0.6  =0.24$|
| 3 | 23 | 24 | $0.6\times 0.4 =0.24$|
| 4 | 23 | 22 | $0.6\times 0.6 =0.36$|

The decision-making problem faced by the producer can be formulated as a three-stage stochastic programming problem. The decision process is as follows:
1. Before the 6-day period the producer needs to decide the quantity to be sold through the contract spanning the whole 6-day period. This is the first-stage decision.
2. For each price realization of the first 3-day period, the producer has to decide both the amount of energy to be sold through the contract spanning the second 3-day period and the amount of energy to be sold in the pool during the first 3-day period. These are the second-stage decisions, which depend on the pool price realization in the first 3-day period.
3. Finally, the producer needs to decide the power to be sold in the pool, which spans the second 3-day period, for each one of the four price com- binations involving first and second 3-day periods.

### Problem Formulation (Scenario-Variable)
To formulate this problem as a multi-stage stocahstic optimization problem, we'll use the following notation:
- $P_{\omega}^c$ for $\omega = 1,2,3,4$ and $c \in \{A,B\}$: power to be sold through contract $c$ in scenario $\omega$
- $P_\omega^t$ for $\omega = 1,2,3,4$ and $t \in \{t_1, t_2\}$: power to be sold in the pool for period $t$ in scenario $\omega$

Using the data above, the LP formulation for this three-stage stochastic model is:

$$
\begin{align*}
\max \quad & \Pi^S = 0.16 \times 72(2\times 25P_1^A + 26P_1^B + 27P_1^{t_1} + 28P_1^{t_2}) & \text{Revenue $\omega = 1$}\\
& \quad +  0.24\times 72(2\times 25P_2^A + 26P_2^B + 27P_2^{t_1} + 26P_2^{t_2}) & \text{Revenue $\omega = 2$}\\
& \quad +  0.24 \times 72(2\times 25P_3^A + 26P_3^B + 23P_3^{t_1} + 24P_3^{t_2}) & \text{Revenue $\omega = 3$}\\
& \quad +  0.36 \times 72(2\times 25P_4^A + 26P_4^B + 23P_4^{t_1} + 22P_4^{t_2}) & \text{Revenue $\omega = 4$}\\
\text{s.t.} \quad & P_1^A + P_1^{t_1} \leq 120 & \text{Prod cap period $t_1$, scenario $\omega=1$} \\
& P_2^A + P_2^{t_1} \leq 120 & \text{Prod cap period $t_1$, scenario $\omega=2$} \\
& P_3^A + P_3^{t_1} \leq 120 & \text{Prod cap period $t_1$, scenario $\omega=3$} \\
& P_4^A + P_4^{t_1} \leq 120 & \text{Prod cap period $t_1$, scenario $\omega=4$} \\
& P_1^A + P_1^B + P_1^{t_2} \leq 120 & \text{Prod cap period $t_2$, scenario $\omega=1$} \\
& P_2^A + P_2^B + P_2^{t_2} \leq 120 & \text{Prod cap period $t_2$, scenario $\omega=2$} \\
& P_3^A + P_3^B + P_3^{t_2} \leq 120 & \text{Prod cap period $t_2$, scenario $\omega=3$} \\
& P_4^A + P_4^B + P_4^{t_2} \leq 120 & \text{Prod cap period $t_2$, scenario $\omega=4$} \\
& 0 \leq P_\omega^A \leq 50 \qquad \omega = 1,2,3,4 & \text{Contract A cap} \\
& 0 \leq P_\omega^B\leq 40 \qquad \omega = 1,2,3,4 & \text{Contract B cap} \\
& P_1^A = P_2^A = P_3^A = P_4^A & \text{Stage 1 nonanticipitivity} \\
& P_1^B = P_2^B,  P_3^B = P_4^B & \text{Stage 2 nonanticipitivity} \\
& P_1^{t_1} = P_2^{t_1},  P_3^{t_1} = P_4^{t_1} & \text{Stage 2 nonanticipitivity} \\
\end{align*}
$$

## Stochastic Model Implementation
We will implement this power market contract model using [`Pyomo`](https://pyomo.readthedocs.io/en/stable/). Run the cells below to formulate and solve the model.

In [ ]:
# initialize Pyomo model object
stoc_model = ConcreteModel(name='Power Market Contract Model')

# declare contract data
stoc_model.contracts = Set(initialize=['A', 'B'])


stoc_model.contract_duration = Param(stoc_model.contracts, # hours
                                    initialize={'A': 144,
                                                'B': 72})

stoc_model.contract_price = Param(stoc_model.contracts, # $/MWh
                                  initialize={'A': 25,
                                              'B': 26},
                                  mutable=True)

stoc_model.contract_capacity = Param(stoc_model.contracts, # MW
                                    initialize={'A': 50,
                                              'B': 40})

stoc_model.prod_capacity = Param(initialize=120) # MW

In [ ]:
# initialize scenario data
num_scenarios = 4


# price data for each (period, scenario) pair
pool_prices = {('t1', 1): 27, ('t1', 2): 27, ('t1', 3): 23, ('t1', 4): 23, # first period
               ('t2', 1): 28, ('t2', 2): 26, ('t2', 3): 24, ('t2', 4): 22} # second period

scenario_probabilities = {1: 0.16, 2: 0.24, 3: 0.24, 4: 0.36}

stoc_model.scenarios = RangeSet(1,num_scenarios)
stoc_model.periods = Set(initialize=['t1', 't2'])

stoc_model.pool_price = Param(stoc_model.periods, stoc_model.scenarios,
                                initialize=pool_prices)

stoc_model.scenario_prob = Param(stoc_model.scenarios,
                                  initialize=scenario_probabilities)


In [ ]:
# declare variables
stoc_model.contract_sale = Var(stoc_model.contracts, stoc_model.scenarios,
                                    within=NonNegativeReals)

stoc_model.pool_sale = Var(stoc_model.periods, stoc_model.scenarios,
                               within=NonNegativeReals)


In [ ]:
# declare constraints

# producer production capacity constraint in each scenario
#   contract sales + pool sales <= 120
def prod_capacity_constraint_(m, period, scenario):

    if period == 't1': # first period
        return m.contract_sale['A', scenario] + m.pool_sale[period,scenario] <= m.prod_capacity
    else: # second period
        return m.contract_sale['A', scenario] + m.contract_sale['B', scenario] \
                + m.pool_sale[period,scenario] <= m.prod_capacity

stoc_model.prod_capacity_constraint = Constraint(stoc_model.periods, stoc_model.scenarios,
                                                 rule=prod_capacity_constraint_)

# contract capacity constraint for each scenario
#   contract sale <= contract cap
def contract_capacity_constraint_(m,contract,scenario):
    return m.contract_sale[contract, scenario] <= m.contract_capacity[contract]

stoc_model.contract_capacity_constraint = Constraint(stoc_model.contracts, stoc_model.scenarios,
                                                     rule=contract_capacity_constraint_)

# stage 1 nonantivipitivity constraints
def stage1_nonanticipitivity_constraints_(m):
    yield m.contract_sale['A', 2] == m.contract_sale['A', 3]

stoc_model.stage1_nonanticipitivity_constraints = ConstraintList(rule=stage1_nonanticipitivity_constraints_)

# stage 2 nonanticipitivity constraints
def stage2_nonanticipitivity_constraints_(m):
    yield m.contract_sale['A', 1] == m.contract_sale['A', 2]
    yield m.contract_sale['A', 3] == m.contract_sale['A', 4]
    yield m.contract_sale['B', 1] == m.contract_sale['B', 2]
    yield m.contract_sale['B', 3] == m.contract_sale['B', 4]
    yield m.pool_sale['t1', 1] == m.pool_sale['t1', 2]
    yield m.pool_sale['t1', 3] == m.pool_sale['t1', 4]

stoc_model.stage2_nonanticipitivity_constraints = ConstraintList(rule=stage2_nonanticipitivity_constraints_)


In [ ]:
# set model objective
def total_revenue_(m):
    obj = 0
    for scenario in m.scenarios:
        revenue = 0

        # revenue from contract sales
        for contract in m.contracts:
            revenue += (m.contract_duration[contract] * m.contract_price[contract])\
                            *m.contract_sale[contract,scenario]
        # revenue from pool sales
        for period in m.periods:
            revenue += (m.contract_duration['B'] * m.pool_price[period,scenario])\
                            *m.pool_sale[period,scenario]
        # expected value
        obj += m.scenario_prob[scenario]*revenue
    return obj
stoc_model.objective = Objective(rule=total_revenue_, sense=maximize)

In [ ]:
# solve model
res = opt.solve(stoc_model, tee=False)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# output model report
stoc_obj_val = stoc_model.objective()
print_stoc_solution(stoc_model)

## Affect of Modified Contract Prices

**Exercise 1 (JN-4):** Now, modify the problem data. Namely, let Contract A's price be any value between 20 and 25 as opposed to just 25.


In [ ]:
new_contract_A_price = 30 # modify me!

Now, run the cell below to solve the problem with modified data.
* Does this change the solution? If so, how does the solution change?
* Is the energy producer better off?

In [ ]:
# create copy of model instance
stoc_model_mod = stoc_model.create_instance()
stoc_model_mod.name = 'Stochastic Model with Modified Data'

# change contract A price
stoc_model_mod.contract_price['A'] = new_contract_A_price

# solve model
res = opt.solve(stoc_model_mod, tee=False)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# output solution
print_stoc_solution(stoc_model_mod)

### Sensitivity Analysis on Contract A Price

Next, we'll conduct sensitivity analysis on the price of Contract A. Specifically, we'll solve the problem for different values of this price between 20 and 30 see how the solution changes. Run the cells below to conduct this experiment.
* How does the optimal solution change as the price of contract A changes?
* At what price does the model solution change significantly?
* Can you derive this point from the problem data?

In [ ]:
# create an array of varying contract prices
contract_prices = np.linspace(20, 30, 20)

# initialize result storing objects
revenues = np.zeros_like(contract_prices)
contract_sales = {'A': {s:np.zeros_like(contract_prices) for s in stoc_model.scenarios},
                      'B': {s:np.zeros_like(contract_prices) for s in stoc_model.scenarios}}
pool_sales = {'t1': {s:np.zeros_like(contract_prices) for s in stoc_model.scenarios},
                  't2': {s:np.zeros_like(contract_prices) for s in stoc_model.scenarios}}

# solve model for varying contract A price
for i, price in np.ndenumerate(contract_prices):

    # create copy of model instance
    stoc_model_temp = stoc_model.create_instance()

    # change contract A price
    stoc_model_temp.contract_price['A'].set_value(price)

    # solve model
    res = opt.solve(stoc_model_temp)

    # check that solver status exited normally
    assert res.solver.status == SolverStatus.ok, \
            f"Solver did not exit normally, termination condition {res.solver.status}."

    # store solution
    revenues[i] = stoc_model_temp.objective()
    for s in stoc_model.scenarios:
        contract_sales['A'][s][i] = stoc_model_temp.contract_sale['A',s].value
        contract_sales['B'][s][i] = stoc_model_temp.contract_sale['B',s].value
        pool_sales['t1'][s][i] = stoc_model_temp.pool_sale['t1',s].value
        pool_sales['t2'][s][i] = stoc_model_temp.pool_sale['t2',s].value



In [ ]:
# plot revenues as a function of contract A price
plt.plot(contract_prices,revenues)
plt.xlabel('Contract A Price ($/MWh)')
plt.ylabel('Expected Revenue ($)')
plt.title('Expected Revenue ($) for Different Values of Contract A Price')
plt.grid()
plt.show()

In [ ]:
# plot contract sales under each scenario
plt.figure(figsize=(12,10))
for s in stoc_model.scenarios:
    plt.subplot(2,2,s)
    plt.plot(contract_prices,contract_sales['A'][s],label='Contract A',color='blue')
    plt.plot(contract_prices,contract_sales['B'][s],label='Contract B',color='red')
    plt.xlabel('Contract A Price ($/MWh)')
    plt.ylabel('Contract Sales (MW)')
    plt.title(f"Contract Sales, Scenario {s}")
    plt.grid()
    plt.legend()

In [ ]:
plt.figure(figsize=(12,10))
for s in stoc_model.scenarios:
    plt.subplot(2,2,s)
    plt.plot(contract_prices,pool_sales['t1'][s],label='Period 1',color='blue')
    plt.plot(contract_prices,pool_sales['t2'][s],label='Period 2',color='red')
    plt.xlabel('Contract A Price ($/MWh)')
    plt.ylabel('Pool Sales (MW)')
    plt.title(f"Pool Sales, Scenario {s}")
    plt.grid()
    plt.legend()

## Removing Nonanticipitivity Constraints
Now we analyze the affect of removing uncertainty from the model. In the original formulation, this uncertainty was implemented by the nonanticipitivity constraints, which enfore that the decisions made at each stage of the model are consistent between scenarios. Since we have a three-stage model, there are two stages of uncertainty and thus two sets of nonanticipitivity constraints. By removing one stages nonanticipitivity constraints, we can determine the optimal decision with perfect information (PI) at that stage.

### Expected Value of Perfect Information
First, will analyze such affects by first looking at the *expected value of perfect information* (EVPI). Define the following:
- $\Pi^{S*}$: optimal value of original three-stage recourse problem
- $\Pi^{P1*}$: optimal value with perfect information in stage 1
- $\Pi^{P2*}$: optimal value with perfect information in stage 1 & 2
The EVPI at each stage is then calcualted as:
$$\text{EVPI}_1 = \Pi^{P2*} - \Pi^{P1*}, \quad \text{and} \quad \text{EVPI}_2 = \Pi^{P1*} - \Pi^{S*}$$
Then the total EVPI for the mult-stage model is
$$ \text{EVPI} = \text{EVPI}_1 + \text{EVPI}_2.$$
Run the cells below to remove the nonanticipitivity constraints at each stage and solve the resulting model.


In [ ]:
print(f"Objective value of stochastic model: ${stoc_obj_val:.2f}")

# make a copy of the model
pi_model = stoc_model.create_instance()

# remove stage 1 nonanticipitivity constraints
del pi_model.stage1_nonanticipitivity_constraints

# solve model with perfect information (PI) in stage 1
res = opt.solve(pi_model,tee=False)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# store objective value
pi_1 = pi_model.objective()
print(f"Objective value with perfect info. in stage 1: ${pi_1:.2f}")

# remove stage 2 nonanticipitivity constraints
del pi_model.stage2_nonanticipitivity_constraints

# solve model with perfect information in stage 1 & 2
res = opt.solve(pi_model,tee=False)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# store objective value
pi_2 = pi_model.objective()
print(f"Objective value with perfect info. in stage 1 & 2: ${pi_2:.2f}")



Now, run the following cell to compute the EVPI.

In [ ]:
# compute expected value of perfect information (EVPI)
evpi_1 = pi_2 - pi_1
evpi_2 = pi_1 - stoc_obj_val
evpi = evpi_1 + evpi_2
print(f"Stage 1 EVPI: ${evpi_1:.2f}")
print(f"Stage 2 EVPI: ${evpi_2:.2f}")
print(f"Total EVPI: ${evpi:.2f}")

**Exercise 2 (JN-5):** Next, modify the price of Contract A's price to be any value in [20, 25] as opposed to just 25.


In [ ]:
new_contract_A_price = 25 # modify me!

Run the cells below to compute the expected value of perfect information (EVPI) for the problem with modified data.
* How does the EVPI change? Is this what you expected?

In [ ]:
# create copy of model instance
stoc_model_mod= stoc_model.create_instance()

# change contract A price
stoc_model_mod.contract_price['A'] = new_contract_A_price

# solve model
res = opt.solve(stoc_model_mod)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

stoc_obj_mod = stoc_model_mod.objective()
print(f"Objective value of stochastic model: ${stoc_obj_mod:.2f}")

# make a copy of the model
pi_model = stoc_model_mod.create_instance()

# remove stage 1 nonanticipitivity constraints
del pi_model.stage1_nonanticipitivity_constraints

# solve model with perfect information (PI) in stage 1
res = opt.solve(pi_model,tee=False)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# store objective value
pi_1 = pi_model.objective()
print(f"Objective value with perfect info. in stage 1: ${pi_1:.2f}")

# remove stage 2 nonanticipitivity constraints
del pi_model.stage2_nonanticipitivity_constraints

# solve model with perfect information in stage 1 & 2
res = opt.solve(pi_model,tee=False)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# store objective value
pi_2 = pi_model.objective()
print(f"Objective value with perfect info. in stage 1 & 2: ${pi_2:.2f}")



In [ ]:
# compute expected value of perfect information (EVPI)
evpi_1 = pi_2 - pi_1
evpi_2 = pi_1 - stoc_obj_mod
evpi = evpi_1 + evpi_2
print(f"Stage 1 EVPI: ${evpi_1:.2f}")
print(f"Stage 2 EVPI: ${evpi_2:.2f}")
print(f"Total EVPI: ${evpi:.2f}")

Now, we'll solve the model for varying contract A prices between 20 and 30 and record the EVPI at each stage. Run the cells below to conduct the experiment and visualize results. Can you relate the critical points in contract A prices to the original problem data?

In [ ]:
contract_prices = np.linspace(20, 30, 20)

evpis = np.zeros_like(contract_prices)
s1_evpis = np.zeros_like(contract_prices)
s2_evpis = np.zeros_like(contract_prices)

for i, price in np.ndenumerate(contract_prices):

    # create a new model instance
    stoc_model_temp = stoc_model.create_instance()

    # set contract price
    stoc_model_temp.contract_price['A'] = price

    # solve original stochastic model
    res = opt.solve(stoc_model_temp)
    imp_obj_val = stoc_model_temp.objective()

    # create copy of model
    pi_model = stoc_model_temp.create_instance()

    # remove stage 1 nonanticipitivity constraints
    del pi_model.stage1_nonanticipitivity_constraints

    # solve model with perfect information (PI) in stage 1
    res = opt.solve(pi_model)

    # store objective value
    pi_1 = pi_model.objective()

    # remove stage 2 nonanticipitivity constraints
    del pi_model.stage2_nonanticipitivity_constraints

    # solve model with perfect information in stage 1 & 2
    res = opt.solve(pi_model)

    # store objective value
    pi_2 = pi_model.objective()
    s1_evpis[i] = pi_2 - pi_1
    s2_evpis[i] = pi_1 - imp_obj_val
    evpis[i] = s1_evpis[i] + s2_evpis[i]




In [ ]:
plt.plot(contract_prices,evpis,label='EVPI')
plt.plot(contract_prices,s1_evpis,label='EVPI Stage 1')
plt.plot(contract_prices,s2_evpis,label='EVPI Stage 2')
plt.xlabel('Contract A Price ($/MWh)')
plt.ylabel('EVPI ($)')
plt.title("Expected Value of Perfect Information (EVPI)")
plt.grid()
plt.legend()
plt.show()

### Value of the Stochastic Solution
Next, we'll analyze the affect of perfect information in the model by looking at the *value of the stochastic solution* (VSS). In the deterministic problem associated with the multi-stage stocahstic program, the random variables are replaced by their respective expected values. The solution to this deterministic problem provides optimal values for the first stage-variables. The original stochastic program can then be solved fixing the values of the first-stage variables to those provided by the deterministic one. To compute the VSS, define the following:
- $\Pi^{S}$: optimal value of the original three-stage recourse problem (as before)
- $\Pi^{D1}$: optimal value with first stage decisions fixed in multi-stage problem
- $\Pi^{D2}$ optimal value with first stage decisions fixed and second stage r.v.'s replaced with averages
The VSS of each stage in the problem is the comptued as
$$ \text{VSS}_{1} = \Pi^{D1} - \Pi^{D2} \quad \text{and} \quad \text{VSS}_2 = \Pi^{S} - \Pi^{D1}.$$
Finally, we can compute the VSS of the multi-stage problem as
$$ \text{VSS} = \text{VSS}_1 + \text{VSS}_2.$$
Run the cells below to formulate the deterministic problem and compute the VSS.

In [ ]:

# create a new model instance
d1_model = stoc_model.create_instance()

# remove all constraints from original multi-stage stochastic model
for con_name, object in d1_model.component_map(Constraint).items():
    delattr(d1_model,con_name)

# remove objective from multi-stage stochastic model
del d1_model.objective

# remove scenario variables for contract A (first stage decision)
for s in d1_model.scenarios:
    del d1_model.contract_sale['A', s]

# add fixed, deterministic contract sale variable for contract A
d1_model.det_contract_A_sale = Param(initialize=50)

# production capacity constraints
def prod_capacity_constraint_d1_(m):

    # first period
    yield m.det_contract_A_sale + m.pool_sale['t1',1] <= m.prod_capacity
    yield m.det_contract_A_sale + m.pool_sale['t1',3] <= m.prod_capacity

    # second period
    yield m.det_contract_A_sale + m.contract_sale['B', 1] + m.pool_sale['t2',1] <= m.prod_capacity
    yield m.det_contract_A_sale + m.contract_sale['B', 1] + m.pool_sale['t2',2] <= m.prod_capacity
    yield m.det_contract_A_sale + m.contract_sale['B', 3] + m.pool_sale['t2',3] <= m.prod_capacity
    yield m.det_contract_A_sale + m.contract_sale['B', 3] + m.pool_sale['t2',4] <= m.prod_capacity

d1_model.prod_capacity_constraint = ConstraintList(rule=prod_capacity_constraint_d1_)

def contract_capacity_constraint_d1_(m):
    yield m.contract_sale['B', 1] <= m.contract_capacity['B']
    yield m.contract_sale['B', 3] <= m.contract_capacity['B']
d1_model.contract_capacity_constraint = ConstraintList(rule=contract_capacity_constraint_d1_)

# deteministic objecrive with first-stage decisions fixed
def total_revenue_d1_(m):

    # revenues from contract A
    obj = (m.contract_duration['A'] * m.contract_price['A'])\
                            * m.det_contract_A_sale
    obj += 0.4*(
                m.contract_duration['B']*(m.contract_price['B']*m.contract_sale['B',1]\
                                          + m.pool_price['t1', 1]*m.pool_sale['t1',1]\
                                          + 0.4*m.pool_price['t2', 1]*m.pool_sale['t2',1]\
                                          + 0.6*m.pool_price['t2', 2]*m.pool_sale['t2',2]
                                         )
                )
    obj += 0.6*(
                m.contract_duration['B']*(m.contract_price['B']*m.contract_sale['B',3]\
                                          + m.pool_price['t1', 3]*m.pool_sale['t1',3]\
                                          + 0.4*m.pool_price['t2', 3]*m.pool_sale['t2',3]\
                                          + 0.6*m.pool_price['t2', 4]*m.pool_sale['t2',4]
                                         )
                )

    return obj
d1_model.objective = Objective(rule=total_revenue_d1_,
                                 sense='maximize')


In [ ]:
# solve model
res = opt.solve(d1_model)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# store objective value
d1_obj_val = d1_model.objective()

Now, we set stage 2 random variables to their mean values with stage 1 decisions fixed and solve the resulting model.

In [ ]:
# create new instance
d2_model = d1_model.create_instance()

# remove all constraints from original multi-stage stochastic model
for con_name, object in d2_model.component_map(Constraint).items():
    delattr(d2_model,con_name)
del d2_model.objective

# remove scenario variables for second stage variables
for s in d2_model.scenarios:
    del d2_model.contract_sale['B', s]
    del d2_model.pool_sale['t1', s]

# add deterministic contract sale variable for contract B
d2_model.det_contract_B_sale = Param(initialize=40)
d2_model.det_t1_pool_sale = Param(initialize=70)

# capacity constraints
def prod_capacity_constraint_d2_(m):

    # second period
    yield m.det_contract_A_sale + m.det_contract_B_sale + m.pool_sale['t2',1] <= m.prod_capacity
    yield m.det_contract_A_sale + m.det_contract_B_sale + m.pool_sale['t2',2] <= m.prod_capacity
    yield m.det_contract_A_sale + m.det_contract_B_sale + m.pool_sale['t2',3] <= m.prod_capacity
    yield m.det_contract_A_sale + m.det_contract_B_sale + m.pool_sale['t2',4] <= m.prod_capacity

d2_model.prod_capacity_constraint = ConstraintList(rule=prod_capacity_constraint_d2_)

# deterministic objective
def total_revenue_d2_(m):

    # revenues from contract A
    obj = (m.contract_duration['A'] * m.contract_price['A'])\
                            * m.det_contract_A_sale
    obj += 0.4*(
                m.contract_duration['B']*(m.contract_price['B']*m.det_contract_B_sale\
                                          + m.pool_price['t1', 1]*m.det_t1_pool_sale\
                                          + 0.4*m.pool_price['t2', 1]*m.pool_sale['t2',1]\
                                          + 0.6*m.pool_price['t2', 2]*m.pool_sale['t2',2]
                                         )
                )
    obj += 0.6*(
                m.contract_duration['B']*(m.contract_price['B']*m.det_contract_B_sale\
                                          + m.pool_price['t1', 3]*m.det_t1_pool_sale\
                                          + 0.4*m.pool_price['t2', 3]*m.pool_sale['t2',3]\
                                          + 0.6*m.pool_price['t2', 4]*m.pool_sale['t2',4]
                                         )
                )

    return obj
d2_model.objective = Objective(rule=total_revenue_d2_,
                                 sense='maximize')


In [ ]:
# solve model
res = opt.solve(d2_model)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# store objective value
d2_obj_val = d2_model.objective()

In [ ]:
vss1 = d1_obj_val - d2_obj_val
print(f"Value of Stochastic Solution Stage 1: {vss1:.2f}")
vss2 = stoc_obj_val - d1_obj_val
print(f"Value of Stochastic Solution Stage 2: {vss2:.2f}")
vss = vss1 + vss2
print(f"Value of Stochastic Solution Stage 1 & 2: {vss:.2f}")


**Exercise 3 (JN-6):** Next, change the price of contract A to be any value betweeen 20 and 25 as opposed to just 25.

In [ ]:
new_contract_A_price = 25 # modify me!

Run the cells below to compute the VSS for the problem with new data.
* How does the solution change? Why?

In [ ]:
# create new model instance
stoc_model_mod = stoc_model.create_instance()

# modify contract price of contract A
stoc_model_mod.contract_price['A'] = new_contract_A_price

# solve modified stochastic model
res = opt.solve(stoc_model_mod)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# store objective
stoc_model_obj_mod = stoc_model_mod.objective()

# create deterministic model and set price
det1_model_mod = d1_model.create_instance()
det1_model_mod.contract_price['A'] = new_contract_A_price

res = opt.solve(det1_model_mod)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# store objective value
d1_obj_val_mod = det1_model_mod.objective()

det2_model_mod = d2_model.create_instance()
det2_model_mod.contract_price['A'] = new_contract_A_price

res = opt.solve(det2_model_mod)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

d2_obj_val_mod = det2_model_mod.objective()

# compute VSS
vss1_mod = d1_obj_val_mod - d2_obj_val_mod
print(f"Value of Stochastic Solution Stage 1: {vss1_mod:.2f}")
vss2_mod = stoc_model_obj_mod - d1_obj_val_mod
print(f"Value of Stochastic Solution Stage 2: {vss2_mod:.2f}")
vss_mod = vss1_mod + vss2_mod
print(f"Value of Stochastic Solution Stage 1 & 2: {vss_mod:.2f}")

Lastly, run the cells below to perform a sensitivity analysis on the affect of varying contract A prices on the VSS.
* What trends do you see?
* How does it relate to the problem data?

In [ ]:
contract_prices = np.linspace(20, 30, 20)

vss = np.zeros_like(contract_prices)
s1_vss = np.zeros_like(contract_prices)
s2_vss = np.zeros_like(contract_prices)

for i, price in np.ndenumerate(contract_prices):

    stoc_model_temp = stoc_model.create_instance()
    stoc_model_temp.contract_price['A'] = price

    res = opt.solve(stoc_model_temp)
    stoc_obj_val_temp = stoc_model_temp.objective()

    det1_model = d1_model.create_instance()
    det1_model.contract_price['A'] = price

    res = opt.solve(det1_model)

    # store objective value
    det1_obj_val = det1_model.objective()

    det2_model = d2_model.create_instance()
    det2_model.contract_price['A'] = price

    res = opt.solve(det2_model)

    det2_obj_value = det2_model.objective()

    # store objective value
    s1_vss[i] = det1_obj_val - det2_obj_value
    s2_vss[i] = stoc_obj_val_temp - det1_obj_val
    vss[i] = s1_vss[i] + s2_vss[i]



In [ ]:
plt.plot(contract_prices,vss,label='VSS')
plt.plot(contract_prices,s1_vss,label='VSS Stage 1')
plt.plot(contract_prices,s2_vss,label='VSS Stage 2')
plt.xlabel('Contract A Price ($/MWh)')
plt.ylabel('VSS ($)')
plt.title("Value of the Stochastic Solution (VSS)")
plt.grid()
plt.legend()
plt.show()